In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
from graphframes import *
from google.cloud import storage
import pickle
import pyspark

In [4]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_ir313254724_322769175_pagerank'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [5]:
parquetFile = spark.read.parquet(*paths)
pages_links = parquetFile.select("id", "anchor_text").rdd

In [6]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # Extract links from Wikipedia articles and flatten the result
  edges_rdd = pages.flatMap(lambda row: ((row.id, anchor.id) for anchor in row.anchor_text))

  # Remove duplicates from the edges RDD
  edges_rdd = edges_rdd.distinct()

  # Create vertices RDD by collecting unique page IDs from edges
  vertices_rdd = (edges_rdd
                  .flatMap(lambda x: (x[0], x[1]))
                  .distinct()
                  .map(lambda x: (x,)))

  return edges_rdd, vertices_rdd

In [7]:
# construct the graph for a small sample of (1000) pages
edges, vertices = generate_graph(pages_links)

# time the actual execution
v_cnt, e_cnt = vertices.count(), edges.count()

edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)

24/03/08 05:55:39 WARN YarnAllocator: Container from a bad node: container_1709834798656_0014_01_000007 on host: project-ir-selin-dior-w-3.c.project-ir-413718.internal. Exit status: 137. Diagnostics: [2024-03-08 05:55:39.382]Container killed on request. Exit code is 137
[2024-03-08 05:55:39.382]Container exited with a non-zero exit code 137. 
[2024-03-08 05:55:39.383]Killed by external signal
.
24/03/08 05:55:39 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 6 for reason Container from a bad node: container_1709834798656_0014_01_000007 on host: project-ir-selin-dior-w-3.c.project-ir-413718.internal. Exit status: 137. Diagnostics: [2024-03-08 05:55:39.382]Container killed on request. Exit code is 137
[2024-03-08 05:55:39.382]Container exited with a non-zero exit code 137. 
[2024-03-08 05:55:39.383]Killed by external signal
.
24/03/08 05:55:39 ERROR YarnScheduler: Lost executor 6 on project-ir-selin-dior-w-3.c.project-ir-413718.internal: Container f

Py4JJavaError: An error occurred while calling o203.run.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 22.0 failed 4 times, most recent failure: Lost task 1.3 in stage 22.0 (TID 827) (project-ir-selin-dior-w-3.c.project-ir-413718.internal executor 18): ExecutorLostFailure (executor 18 exited caused by one of the running tasks) Reason: Container from a bad node: container_1709834798656_0014_01_000019 on host: project-ir-selin-dior-w-3.c.project-ir-413718.internal. Exit status: 137. Diagnostics: [2024-03-08 06:24:27.912]Container killed on request. Exit code is 137
[2024-03-08 06:24:27.912]Container exited with a non-zero exit code 137. 
[2024-03-08 06:24:27.912]Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2717)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2653)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abor

In [ ]:
pr = pr_results.vertices.select("id", "pagerank")

In [ ]:
pr_dict = pr.rdd.map(lambda row: (row.id, row.pagerank)).collectAsMap()

In [ ]:
name = "pagerank_dict"
with open(f"{name}.pkl", "wb") as f:
    pickle.dump(pr_dict, f)

client = storage.Client()
bucket = client.bucket(bucket_name)
blob_posting_locs = bucket.blob(f"page_rank/{name}.pkl")
blob_posting_locs.upload_from_filename(f"{name}.pkl")